In [12]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from db import obtener_stock_producto

import numpy as np


In [13]:
model = SentenceTransformer('all-MiniLM-L6-v2')


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [16]:
funciones = [
    {
        "nombre": "buscar_producto",
        "ejemplos": [
            "¿Cuántas lámparas hay disponibles?",
            "¿Cuántos productos hay en stock?",
            "¿Qué precio tiene la lámpara?",
            "Muéstrame el inventario de mesas",
            "¿Hay sillas disponibles?",
            "Consultar producto lámpara",
            "Ver disponibilidad del producto"
        ]
    },
    {
        "nombre": "crear_pedido",
        "ejemplos": [
            "Quiero comprar una lámpara",
            "Deseo adquirir un producto",
            "Quiero hacer un pedido",
            "Pedir una mesa",
            "Ordenar productos",
            "Realizar una compra"
        ]
    },
    {
        "nombre": "buscar_cliente",
        "ejemplos": [
            "Muéstrame los datos del cliente Juan Pérez",
            "Buscar cliente",
            "Consultar información del cliente",
            "Datos del cliente",
            "Ver cliente registrado"
        ]
    },
    {
        "nombre": "confirmar_pedido",
        "ejemplos": [
            "Sí, confirma el pedido",
            "Confirmar compra",
            "Finalizar pedido",
            "Aceptar pedido",
            "Confirmar orden"
        ]
    }
]


In [17]:
from sklearn.metrics.pairwise import cosine_similarity


# Crear lista plana de ejemplos
frases = []
mapa_funciones = []

for f in funciones:
    for ejemplo in f["ejemplos"]:
        frases.append(ejemplo)
        mapa_funciones.append(f["nombre"])

embeddings_funciones = model.encode(frases)


def seleccionar_funcion(query, umbral=0.45):
    query_embedding = model.encode([query])
    scores = cosine_similarity(query_embedding, embeddings_funciones)[0]

    idx = scores.argmax()
    max_score = scores[idx]

    if max_score < umbral:
        return "conversacion", max_score

    return mapa_funciones[idx], max_score



In [18]:
def ejecutar_funcion(funcion, query):
    query_lower = query.lower()

    if funcion == "conversacion":
        return "Hola 👋 Soy el asistente de ChatWorks. Puedo ayudarte a consultar stock, gestionar clientes o realizar pedidos."

    elif funcion == "buscar_producto":
        nombre, stock, precio = obtener_stock_producto("lampara")

        if nombre:
            return f"El producto {nombre} tiene {stock} unidades disponibles y cuesta ${precio}."
        else:
            return "No encontré ese producto en el inventario."

    elif funcion == "crear_pedido":
        return "¡Excelente elección! He generado el borrador de tu pedido. ¿Deseas confirmarlo ahora?"

    elif funcion == "buscar_cliente":
        nombre = "el cliente solicitado"
        if "juan" in query_lower:
            nombre = "Juan Pérez"

        return f"Consultando base de datos... He encontrado el perfil de {nombre} con historial activo."

    elif funcion == "confirmar_pedido":
        return "Procesando... ✅ Tu pedido ha sido confirmado y registrado con éxito. ¡Gracias por tu compra!"


In [19]:
query = "¿Cuántas lámparas hay disponibles?"

funcion, score = seleccionar_funcion(query)
respuesta = ejecutar_funcion(funcion, query)

print("Usuario:", query)
print("Sistema:", respuesta)
print("Score de similitud:", score)



Usuario: ¿Cuántas lámparas hay disponibles?
Sistema: No encontré ese producto en el inventario.
Score de similitud: 1.0
